In [ ]:
def somerror(air,random_seed,a,b):
    #a=som_shape[0],b=som_shape[1]
    %matplotlib inline
    %autoreload 2

    from minisom import MiniSom

    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec
    import time
    import pandas as pd
    from mpl_toolkits.basemap import Basemap 
    import matplotlib.pyplot as plt  

    #input dataset
    filename='daily_%s_2019.csv' % (air)
    names=['Latitude','Longitude','Date Local','Arithmetic Mean']
    dataset=pd.read_csv(filename,names=names)
    dataset101=dataset.drop_duplicates(subset=['Latitude'])
    dataset1=dataset.pivot_table(index = 'Date Local',columns = 'Latitude',values = 'Arithmetic Mean')#transform long data format into wide format
    dataset11=dataset1.dropna(axis=1, how='any')#delete sites which includes NaN data
    data=dataset11.values.T
    

    #get information of latitudes and longitudes as lists
    latitudes=dataset11.columns.values.tolist()
    longitudes=[]
    for i in range(len(latitudes)):
        long1=dataset101.Longitude[dataset101['Latitude'] == latitudes[i]].tolist()
        long2=float(long1[0])
        longitudes.append(long2)
    from tslearn.clustering import TimeSeriesKMeans
    from sklearn.preprocessing import StandardScaler
    import matplotlib.pyplot as plt
    X_train = data
    scaler = StandardScaler()
    data = scaler.fit_transform(X_train)#normalized
    
    #som training
    start=time.time()
    som_shape = (a, b)
    n_cluster=int(som_shape[0])*int(som_shape[1])
    som = MiniSom(som_shape[0], som_shape[1], data.shape[1], sigma=0.5, learning_rate=0.5, 
                  neighborhood_function='gaussian', random_seed=random_seed)
    som.pca_weights_init(data)
    print("Training...")
    som.train_batch(data, 50000, verbose=True)  # random training
    print("\n...ready!")


    max_iter = 50000
    q_error = []
    t_error = []

    for i in range(max_iter):
        rand_i = np.random.randint(len(data))
        som.update(data[rand_i], som.winner(data[rand_i]), i, max_iter)
        q_error.append(som.quantization_error(data))
        t_error.append(som.topographic_error(data))

    plt.plot(np.arange(max_iter), q_error, label='quantization error')
    plt.plot(np.arange(max_iter), t_error, label='topographic error')
    plt.ylabel('quantization error')
    plt.xlabel('iteration index')
    plt.legend()
    plt.show()
    
    #visualization-draw the time seires
    win_map = som.win_map(data)
    plt.figure(figsize=(64, 16))
    the_grid = GridSpec(8, 8)
    print(win_map.keys)
    for position in win_map.keys():
        plt.subplot(the_grid[5-position[1], position[0]])
        plt.plot(np.min(win_map[position], axis=0), color='gray', alpha=.5)
        plt.plot(np.mean(win_map[position], axis=0))
        plt.plot(np.max(win_map[position], axis=0), color='gray', alpha=.5)
    plt.title('timeseries-2019,%s,SOM,2*2-seed %d'% (air,random_seed))
    plt.show()

    #visualization-draw the map
    # reference: https://github.com/JustGlowing/minisom/blob/master/examples/Clustering.ipynb
    # each neuron represents a cluster
    winner_coordinates = np.array([som.winner(x) for x in data]).T
    # with np.ravel_multi_index we convert the bidimensional coordinates to a monodimensional index
    cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)


    #draw the US map
    fig=plt.figure(figsize=(7.6, 3.8)) 
    m= Basemap(llcrnrlon=-130, llcrnrlat=20, urcrnrlon=-65, urcrnrlat=49,projection='lcc',lat_1=24, lat_2=45, lon_0=-100)
    m.drawcountries(linewidth=1.5) 
    m.drawcoastlines()  
    m.drawstates()       
    #draw the datapoints
    color=['#FFFF00','#00BFFF','#4B0082','#8B0000','#FFA500', '#EE82EE']
    for j in range(n_cluster):
        lons=[]
        lats=[]
        for i in range(len(cluster_index)):
            if cluster_index[i]==j:
                lons.append(longitudes[i])
                lats.append(latitudes[i])
        lon, lat = m(lons,lats)
        m.scatter(lon,lat,c=color[j],label='Cluster %d' % (j + 1),s=8)
    plt.legend()
    plt.title('map-2019,%s,SOM,2*2-seed %d'% (air,random_seed))
    plt.show()


    end=time.time()
    print("time spending(seconds):")
    print(end-start)


In [ ]:
somerror('O3',10,2,3)

In [ ]:
def som(air,random_seed,a,b):
    #a=som_shape[0],b=som_shape[1]
    %matplotlib inline
    %autoreload 2

    from minisom import MiniSom

    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.gridspec import GridSpec
    import time
    import pandas as pd
    from mpl_toolkits.basemap import Basemap 
    import matplotlib.pyplot as plt  

    #input dataset
    filename='daily_%s_2019.csv' % (air)
    names=['Latitude','Longitude','Date Local','Arithmetic Mean']
    dataset=pd.read_csv(filename,names=names)
    dataset101=dataset.drop_duplicates(subset=['Latitude'])
    dataset1=dataset.pivot_table(index = 'Date Local',columns = 'Latitude',values = 'Arithmetic Mean')#transform long data format into wide format
    dataset11=dataset1.dropna(axis=1, how='any')#delete sites which includes NaN data
    data=dataset11.values.T
    

    #get information of latitudes and longitudes as lists
    latitudes=dataset11.columns.values.tolist()
    longitudes=[]
    for i in range(len(latitudes)):
        long1=dataset101.Longitude[dataset101['Latitude'] == latitudes[i]].tolist()
        long2=float(long1[0])
        longitudes.append(long2)
    from tslearn.clustering import TimeSeriesKMeans
    from sklearn.preprocessing import StandardScaler
    import matplotlib.pyplot as plt
    X_train = data
    scaler = StandardScaler()
    data = scaler.fit_transform(X_train)#normalized
    
    #som training
    start=time.time()
    som_shape = (a, b)
    n_cluster=int(som_shape[0])*int(som_shape[1])
    som = MiniSom(som_shape[0], som_shape[1], data.shape[1], sigma=0.5, learning_rate=0.5, 
                  neighborhood_function='gaussian', random_seed=random_seed)
    som.pca_weights_init(data)
    print("Training...")
    som.train_batch(data, 20000, verbose=True)  # random training
    print("\n...ready!")
    #visualization-draw the time seires
    win_map = som.win_map(data)
    plt.figure(figsize=(64, 16))
    the_grid = GridSpec(8, 8)
    print(win_map.keys)
    for position in win_map.keys():
        plt.subplot(the_grid[5-position[1], position[0]])
        plt.plot(np.min(win_map[position], axis=0), color='gray', alpha=.5)
        plt.plot(np.mean(win_map[position], axis=0))
        plt.plot(np.max(win_map[position], axis=0), color='gray', alpha=.5)
    plt.title('timeseries-2019,%s,SOM,2*2-seed %d'% (air,random_seed))
    plt.show()

    #visualization-draw the map
    # reference: https://github.com/JustGlowing/minisom/blob/master/examples/Clustering.ipynb
    # each neuron represents a cluster
    winner_coordinates = np.array([som.winner(x) for x in data]).T
    # with np.ravel_multi_index we convert the bidimensional coordinates to a monodimensional index
    cluster_index = np.ravel_multi_index(winner_coordinates, som_shape)


    #draw the US map
    fig=plt.figure(figsize=(7.6, 3.8)) 
    m= Basemap(llcrnrlon=-130, llcrnrlat=20, urcrnrlon=-65, urcrnrlat=49,projection='lcc',lat_1=24, lat_2=45, lon_0=-100)
    m.drawcountries(linewidth=1.5) 
    m.drawcoastlines()  
    m.drawstates()       
    #draw the datapoints
    color=['#FFFF00','#00BFFF','#4B0082','#8B0000','#FFA500', '#EE82EE']
    for j in range(n_cluster):
        lons=[]
        lats=[]
        for i in range(len(cluster_index)):
            if cluster_index[i]==j:
                lons.append(longitudes[i])
                lats.append(latitudes[i])
        lon, lat = m(lons,lats)
        m.scatter(lon,lat,c=color[j],label='Cluster %d' % (j + 1),s=8)
    plt.legend()
    plt.title('map-2019,%s,SOM,2*2-seed %d'% (air,random_seed))
    plt.show()


    end=time.time()
    print("time spending(seconds):")
    print(end-start)

In [ ]:
som('O3',10,2,2)